*YOLO is a deep learning algorythm. So it doesn’t need any installation, instead it needs a deep learning freamwork to run on!*

*Three most know used freamwork compatible with YOLO are -* <br/>
*1. Darknet* <br/>
*2. Darkflow* <br/>
*3. Opencv*

*Imports necessary packages*

In [1]:
import numpy as np
import cv2
import os

*Load the COCO class labels our YOLO model was trained on*

In [2]:
lblPath = "./models/coco.names"
LABELS = open(lblPath).read().strip().split("\n")

for i in range(3):
    print(LABELS[i])

person
bicycle
car


*Initialize a list of colors to represent each possible class label*

In [3]:
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

*Load our YOLO object detector model trained on COCO dataset (80 classes). To run the algorythm we need 2 files:*

*1. Weight file: it’s the trained model, the core of the algorythm to detect the objects.* <br/>
*2. Cfg file: it’s the configuration file, where there are all the settings of the algorythm.* <br/>

*Determine only the output layer names that we need from YOLO*

In [4]:
weightsPath = "./models/yolov3.weights"
configPath = "./models/yolov3.cfg"

net = cv2.dnn.readNetFromDarknet(configPath,weightsPath)

out_layers = net.getLayerNames()
out_layers = [out_layers[i[0] - 1] for i in net.getUnconnectedOutLayers()]

*Load our input image and grab its dimensions*

In [13]:
imgPath = ".\images\imgg3.jpg"

image = cv2.imread(imgPath)
H, W, channels = image.shape #(H, W) = image.shape[:2]

*Now that we have the algorythm and image, it’s time to pass the image into the network and do the detection.*

*Keep in mind that we can’t pass the full image on the network. First we need to convert it to a blob. Blob extracts the features from an image and resize them. YOLO accepts three sizes:*

*1.320×320 it’s small so less accuracy but better speed* <br/>
*2.609×609 it’s bigger so high accuracy and slow speed* <br/>
*3.416×416 it’s in the middle and you get a bit of both.* <br/>

*The 'layerOutputs' is the result of the detection. It is an array that conains all the informations about objects detected, their position and the confidence about the detection.*

In [14]:
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)

layerOutputs = net.forward(out_layers)
type(layerOutputs)

list

*Initialize our lists of detected bounding boxes, confidences, and class IDs, respectively*

In [15]:
boxes = []
confidences = []
classIDs = []

*1. Loop over each of the layer outputs* <br/>
*2. Filter out weak predictions by ensuring the detected. Probability is greater than the minimum probability* <br/>
*3. Scale the bounding box coordinates back relative to the size of the image, keeping in mind that YOLO actually returns bounding box co-ordinates in the form of (centerX, centerY, width, and height)* <br/>
*4. Use the information to derive the top and left corner of the bounding box* <br/>
*5. Appending list of bounding box coordinates, confidences and class IDs for each object detected* <br/>
*6. Apply non-maxima suppression to suppress weak, overlapping bounding boxes* <br/>
*7. Extract the bounding box coordinates*

In [16]:

for output in layerOutputs:
    for detection in output:
        
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        #2
        if confidence > 0.5:
            #3
            (centerX, centerY, width, height) = (detection[0:4] * np.array([W, H, W, H])).astype("int")
            
            #4
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))

            #5
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            classIDs.append(classID)

#6
idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# ensure at least one detection exists
if len(idxs) > 0:
    for i in idxs.flatten():
        #7
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])

        # draw a bounding box rectangle and label on the image
        color = [int(c) for c in COLORS[classIDs[i]]]
        
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        text = str(LABELS[classIDs[i]]) + " : " + str(confidences[i])
        cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# show the output image
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()